# Chapter 3- Advanced Preprocessing Techniques

## Merging DataFrames

A critical aspect in data analysis and feature engineering is combining multiple datasets because more often, it's spread across several sources, waiting to be collected, organized, and analyzed. Let's learn how to use this powerful tool to combine DataFrames and discover the various merge operations and their usage in different scenarios.

In [1]:
import pandas as pd

df_books = pd.DataFrame({
     "Book_ID": [1, 2, 3, 4, 5],
     "Book_Title": ['Gatsby', 'Mockingbird', '1984', 'Catcher', 'LOTR'],
     "Author_ID": [101, 102, 103, None, 112],
     "Genre": ['Fiction', 'Fiction', 'Fiction', 'Fiction', 'Fantasy']
 })

# creating the DataFrame for Authors
df_authors = pd.DataFrame({
    "Author_ID": [101, 102, 103, 104, 105],
    "Author_Name": ['F. Fitzgerald', 'H. Lee', 'G. Orwell', 'J. Salinger', 'J. Tolkien'],
    "Nationality": ['American', 'American', 'British', 'American', 'British']
})

Two important things to note:

- The author with Author_ID=112 is missing in the df_authors dataframe
- The book named Catcher in the df_books dataframe misses info about its author


In [2]:
# inner join will have only rows with common Author_ID in both dataframes, so we don't include books where author information is missing or undefined.
merged_df = df_books.merge(df_authors, on="Author_ID", how="inner")
print(merged_df)

   Book_ID   Book_Title  Author_ID    Genre    Author_Name Nationality
0        1       Gatsby      101.0  Fiction  F. Fitzgerald    American
1        2  Mockingbird      102.0  Fiction         H. Lee    American
2        3         1984      103.0  Fiction      G. Orwell     British


In [3]:
# outer join includes all the rows from both DataFrames and fills NaN for missing values
merged_df = df_books.merge(df_authors, on="Author_ID", how="outer")
print(merged_df)

   Book_ID   Book_Title  Author_ID    Genre    Author_Name Nationality
0      1.0       Gatsby      101.0  Fiction  F. Fitzgerald    American
1      2.0  Mockingbird      102.0  Fiction         H. Lee    American
2      3.0         1984      103.0  Fiction      G. Orwell     British
3      NaN          NaN      104.0      NaN    J. Salinger    American
4      NaN          NaN      105.0      NaN     J. Tolkien     British
5      5.0         LOTR      112.0  Fantasy            NaN         NaN
6      4.0      Catcher        NaN  Fiction            NaN         NaN


In [4]:
# left join includes all rows from the first DataFrame and fills NaN for missing values in the second DataFrame
merged_df = df_books.merge(df_authors, on="Author_ID", how="left")
print(merged_df)

   Book_ID   Book_Title  Author_ID    Genre    Author_Name Nationality
0        1       Gatsby      101.0  Fiction  F. Fitzgerald    American
1        2  Mockingbird      102.0  Fiction         H. Lee    American
2        3         1984      103.0  Fiction      G. Orwell     British
3        4      Catcher        NaN  Fiction            NaN         NaN
4        5         LOTR      112.0  Fantasy            NaN         NaN


In [5]:
# right join includes all rows from the second DataFrame, in reverse to a left join.
merged_df = df_books.merge(df_authors, on="Author_ID", how="right")
print(merged_df)

   Book_ID   Book_Title  Author_ID    Genre    Author_Name Nationality
0      1.0       Gatsby      101.0  Fiction  F. Fitzgerald    American
1      2.0  Mockingbird      102.0  Fiction         H. Lee    American
2      3.0         1984      103.0  Fiction      G. Orwell     British
3      NaN          NaN      104.0      NaN    J. Salinger    American
4      NaN          NaN      105.0      NaN     J. Tolkien     British


## Concat DataFrames

ignore_index=True parameter? When set to True, it resets the index in the resulting DataFrame. So, in the resultant DataFrame, the indices are in increasing order starting from 0.

In [6]:
import pandas as pd

# df1
data_dict = {"Name": ["John", "Anna", "Peter"],
             "Age": [28, 24, 33],
             "City": ["New York", "Los Angeles", "Berlin"]}

df1 = pd.DataFrame(data_dict)
isYouthful = lambda age: "Yes" if age < 30 else "No"
df1["IsYouthful"] = df1["Age"].apply(isYouthful)

# df2
df2 = pd.DataFrame({"Name": ["Megan"], "Age": [34], "City": ["San Francisco"], "IsYouthful": ["No"]})

# concat df1 and df2
df_concatenated = pd.concat([df1, df2], ignore_index=True)

print(df_concatenated)

    Name  Age           City IsYouthful
0   John   28       New York        Yes
1   Anna   24    Los Angeles        Yes
2  Peter   33         Berlin         No
3  Megan   34  San Francisco         No


# Grouping in Pandas

Grouping rows of a DataFrame is a powerful tool that allows you to aggregate rows based on the values in one or more columns of your data. The groupby function in pandas is the basis of group operations.

In [7]:
import pandas as pd
df = pd.DataFrame({
    "Name": ["Alex", "Bob", "Chloe", "Charlie", "Alex", "Charlie"],
    "Age": [12, 15, 28, 55, 21, 35],
    "City": ["New York", "Los Angeles", "Chicago", "Los Angeles", "New York", "New York"]
})

# the grouped variable is a special pandas DataFrameGroupBy object that has divided our DataFrame into groups by city. It's like a dictionary: 
# each key is a unique city from our City column, and the corresponding value for each key is a DataFrame comprising all rows with that city in the City column.
grouped = df.groupby("City")

print(grouped.get_group("New York"))

      Name  Age      City
0     Alex   12  New York
4     Alex   21  New York
5  Charlie   35  New York


In [8]:
import pandas as pd
# Create a simple dataframe
data = {'Company': ['GOOG', 'GOOG', 'MSFT', 'MSFT', 'FB', 'FB'],
       'Person': ['Sam', 'Charlie', 'Amy', 'Vanessa', 'Carl', 'Sarah'],
       'Sales': [200, 120, 340, 124, 243, 350]}

df = pd.DataFrame(data)

# Apply groupby
df_grouped = df.groupby('Company')

for key, item in df_grouped:
    print("\nGroup Key: {}".format(key))
    print(item)
    # print(df_grouped.get_group(key), "\n")


Group Key: FB
  Company Person  Sales
4      FB   Carl    243
5      FB  Sarah    350

Group Key: GOOG
  Company   Person  Sales
0    GOOG      Sam    200
1    GOOG  Charlie    120

Group Key: MSFT
  Company   Person  Sales
2    MSFT      Amy    340
3    MSFT  Vanessa    124


The  benefit of the groupby method is the variety of operations we can perform on the groupby object. Functions like sum(), mean(), etc., help us simplify the grouped data into more insightful information. Here's how we can use groupby and find out the total sales for each company

In [9]:
grouped = df.groupby('Company')
print(grouped.sum())

             Person  Sales
Company                   
FB        CarlSarah    593
GOOG     SamCharlie    320
MSFT     AmyVanessa    464


## Groupby(), apply() and lambda fn

In [10]:
# using apply() on groupby object
print(df.groupby('Company').apply(lambda x: x['Sales'].max()))

Company
FB      350
GOOG    200
MSFT    340
dtype: int64


/var/folders/ll/1x3pcpw96cn3pc3l3hqztlvr0000gn/T/ipykernel_157/4292066780.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(df.groupby('Company').apply(lambda x: x['Sales'].max()))


In [11]:
# same thing as above
grouped = df.groupby('Company')
print(grouped.max())

          Person  Sales
Company                
FB         Sarah    350
GOOG         Sam    200
MSFT     Vanessa    340


## Applying Aggregation Functions after Grouping or Iterating Through Groups

In [12]:
print(grouped['Age'].mean())

for name, group in grouped:
    print("\nCity:", name)
    print("Number of people:", len(group))
    print("Average age:", group["Age"].mean())

KeyError: 'Column not found: Age'

# Filtering Grouped DataFrame Using Boolean Lambda Function

Boolean selection does not apply to grouped dataframes. Instead, we use the filter() function, which takes a boolean function as an argument. For instance, let's keep products with a summary quantity greater than 90.

In [8]:
import pandas as pd

sales = pd.DataFrame({
  'Product': ['Apple', 'Banana', 'Pear', 'Apple', 'Banana', 'Pear'],
  'Store': ['Store1', 'Store1', 'Store1', 'Store2', 'Store2', 'Store2'],
  'Quantity': [20, 30, 40, 50, 60, 70]
})

grouped = sales.groupby('Product')

print(grouped.get_group('Apple'))

  Product   Store  Quantity
0   Apple  Store1        20
3   Apple  Store2        50


In [ ]:
grouped = sales.groupby('Product')

# filtering using boolean lambda fn
filtered_df = grouped.filter(lambda x: x['Quantity'].sum() > 90)

print(filtered_df)

  Product   Store  Quantity
2    Pear  Store1        40
5    Pear  Store2        70


# Master Sorting

Let us see how to use Panda's sort_values() function for single and multi-column sorting and how to handle missing values.

In [1]:
import pandas as pd

df = pd.DataFrame({
    'Player': ['L. James', 'K. Durant', 'M. Jordan',  'S. Curry', 'K. Bryant'],
    'Points': [27.0, 26.0, 32.0, 24.0, 26.0],
    'Assists': [5.7, 4.7, 4.2, 6.6, 7.4]
})

sorted_df = df.sort_values(by='Points', ascending=False)
print(sorted_df)

      Player  Points  Assists
2  M. Jordan    32.0      4.2
0   L. James    27.0      5.7
1  K. Durant    26.0      4.7
4  K. Bryant    26.0      7.4
3   S. Curry    24.0      6.6


## Sorting on multiple columns

In [2]:
# sorting on multiple columns
sorted_df = df.sort_values(by=['Points', 'Assists'], ascending=False)
print(sorted_df)

      Player  Points  Assists
2  M. Jordan    32.0      4.2
0   L. James    27.0      5.7
4  K. Bryant    26.0      7.4
1  K. Durant    26.0      4.7
3   S. Curry    24.0      6.6


## Sorting on multiple columns in different order

In [3]:
sorted_df = df.sort_values(by=['Points', 'Player'], ascending=[False, True])
print(sorted_df)

      Player  Points  Assists
2  M. Jordan    32.0      4.2
0   L. James    27.0      5.7
4  K. Bryant    26.0      7.4
1  K. Durant    26.0      4.7
3   S. Curry    24.0      6.6


# Random Quote

"It is wise to be cheerful"

"Careful" or any positive word might also fit well in the quote above.